# Week8(CNN_Basic) Assignment2 - Tobigs14 이혜린

AlexNet 모델을 pytorch를 이용하여 구현해보았습니다!  

### Main features

* 구조 : 5 Convolution layers (with 3 MaxPool layers) + 3 fully connected layers  
* activation function : ReLU  
* 2nd, 4th, 5th convolution layer에 weight initializer (평균 0, se 0.01의 gaussian dist)  
* 1st, 2nd fully connected layer에 weight initializer (constant1)  
* 1st, 2nd fully connected layer에 dropout(0.5) 

In [5]:
import torch.nn as nn

In [16]:
class AlexNet(torch.nn.Module):
    
    def __init__(self, num_classes=1000):
        super(__class__, self).__init__()
        
        ### nn.Conv2d(input_channel, output_channel, kernel_size, stride, padding)
        ### nn.MaxPool2d(kernel_size, stride, padding)
        
        conv1 = nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=2)      #layer1 (Convolution layer1)
        pool1 = nn.MaxPool2d(kernel_size=3, stride=2)                      #layer2 (MaxPool layer1)

        conv2 = nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2)     #layer3 (Convolution layer2)
        pool2 = nn.MaxPool2d(kernel_size=3, stride=2)                      #layer4 (MaxPool layer2)

        conv3 = nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1)    #layer5 (Convolution layer3)

        conv4 = nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1)    #layer6 (Convolution layer4)

        conv5 = nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1)    #layer7 (Convolution layer5)
        pool3 = nn.MaxPool2d(kernel_size=3, stride=2)                      #layer8 (MaxPool layer)
        
        lin1 = nn.Linear(256*6*6, 4096)                                    #layer9 (Fully connected layer1)
        lin2 = nn.Linear(4096, 4096)                                       #layer10 (Fully connected layer2)
        lin3 = nn.Linear(4096, num_classes)                                #layer11 (Fully connected layer3)
        
        relu = nn.ReLU(inplace=True)
        
        
        # weight initializer
        nn.init.normal_(conv2.weight, mean=0.0, std=0.01)
        nn.init.normal_(conv4.weight, mean=0.0, std=0.01)
        nn.init.normal_(conv5.weight, mean=0.0, std=0.01)
        nn.init.ones_(lin1.weight)
        nn.init.ones_(lin2.weight)
        
        self.features = nn.Sequential(conv1, relu, pool1, conv2, relu, pool2, 
                                      conv3, relu, conv4, relu, conv5, relu, pool3)
        
        self.classifier = nn.Sequential(nn.Dropout(0.5), lin1, relu, nn.Dropout(0.5), lin2, relu, lin3)
        
        def forward(self, x):
            x = self.features(x)
            x = x.view(x.size(0), 256*6*6)
            x = self.classifier(x)
            return x
        
def alexNet(num_output_classes):
    
    model = AlexNet(num_output_classes)
    
    return model

In [20]:
model = alexNet(1000)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0005 )

# 참고로 논문에선 batch size를 128 로 뒀습니다!